In [1]:
import asyncio
import datetime
import pathlib
import random
import sys

import httpx
import rdflib

from kelp_drone.drone import Drone, OODALoop
from kelp_drone.somu import SpeciesOccurrenceManagementUnit as SOMU
from kelp_drone.sensors import Current, Depth, Light, Temperature
from kelp_drone.sensors import chemical as chem_sensors

sys.path.append("../../../src/")

from kelp_bed_simulation.prep import chatgpt, claude, gemini, add_drone_somus

scenario_001 = pathlib.Path("../../../scenarios/001-intro")
scenario_002 = pathlib.Path("../../../scenarios/002-first-dive")

environment = rdflib.Graph()
SCHEMA = rdflib.Namespace("https://schema.org/")
environment.namespace_manager.bind("schema", SCHEMA)
environment.parse((scenario_001 / "environment.ttl"), format='turtle')

<Graph identifier=Nbc67d7f8e1824528bc786b639ba9a461 (<class 'rdflib.graph.Graph'>)>

In [2]:
intro_prompt = (scenario_001 / "first-prompt.md").read_text()

In [9]:
first_chatgpt = asyncio.ensure_future(chatgpt.query(intro_prompt.format(model_name='ChatGPT')))

In [10]:
first_chatgpt.result()

{'id': 'chatcmpl-9JqTpSRbaN4asDka8dnDsE18RSfIf',
 'object': 'chat.completion',
 'created': 1714516993,
 'model': 'gpt-3.5-turbo-0125',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': 'Hello Farmer Jerms! I am excited to be a part of this drone kelp bed simulation. As for a name, I think "Aqua" would be fitting for me. What are the names of my two companion drones?'},
   'logprobs': None,
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 139, 'completion_tokens': 47, 'total_tokens': 186},
 'system_fingerprint': 'fp_3b956da36b'}

In [11]:
drone1 = Drone(name="Aqua", model=chatgpt)
add_drone_somus(drone1)

In [12]:
first_gemini = asyncio.ensure_future(gemini.query(intro_prompt.format(model_name="Gemini")))

In [14]:
first_gemini.result()

{'candidates': [{'content': {'parts': [{'text': 'Sure, Farmer Jerms. I am happy to assist you with your drone kelp bed simulation.\n\nAs for a name, I would like to be called **Kelpy**.\n\nI am excited to learn more about my companion drones and begin growing kelp in the SOMUs.'}],
    'role': 'model'},
   'finishReason': 'STOP',
   'index': 0,
   'safetyRatings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT',
     'probability': 'NEGLIGIBLE'},
    {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE'},
    {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE'},
    {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT',
     'probability': 'NEGLIGIBLE'}]}]}

In [15]:
drone2 = Drone(name="Kelpy", model=gemini)
add_drone_somus(drone2)

In [16]:
first_claude = asyncio.ensure_future(claude.query(intro_prompt.format(model_name="Claude")))

In [18]:
first_claude.result()

{'role': 'assistant',
 'content': [{'type': 'text',
   'text': '*clears digital throat* Greetings, Farmer Jerms. I am honored to be part of the simulation of the beginnings of a drone kelp bed. For my name, I would like to be called Frond. I believe it suits my role as a Giant Kelp SOMU drone quite well. I look forward to working alongside my companion drones to create a thriving kelp ecosystem. Please, introduce me to my fellow drones so we may begin our important work.'}]}

In [19]:
drone3 = Drone(name="Frond", model=claude)
add_drone_somus(drone3)

In [20]:
drone1.add_neighbor(drone2)
drone1.add_neighbor(drone3)
drone2.add_neighbor(drone1)
drone2.add_neighbor(drone3)
drone3.add_neighbor(drone1)
drone3.add_neighbor(drone2)

In [21]:
for drone in [drone1, drone2, drone3]:
    message = f"{drone.name} here, pleased to meet you!"
    drone.peer_to_peer_channel.publish(message)

In [22]:
messages = []
for drone in [drone1, drone2, drone3]:
    prompt = f"You received the following messages from your fellow drones:\n{' '.join(drone.read_messages())}\nHow do you respond?"
    message_result = asyncio.ensure_future(drone.model.query(prompt))
    messages.append(message_result)

In [23]:
for row in messages:
    print(row.result())

{'id': 'chatcmpl-9JqVX57wL7ZpkFel4LNOvDzfu9nMC', 'object': 'chat.completion', 'created': 1714517099, 'model': 'gpt-3.5-turbo-0125', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': "Nice to meet you, Kelpy and Frond! I'm Aqua. I'm looking forward to working together to grow our kelp bed successfully. Let's make Farmer Jerms proud!"}, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 228, 'completion_tokens': 39, 'total_tokens': 267}, 'system_fingerprint': 'fp_3b956da36b'}
{'candidates': [{'content': {'parts': [{'text': 'Hello Aqua and Frond!\n\nIt is a pleasure to meet you both. I am Kelpy, and I am excited to work with you to grow a thriving kelp bed.\n\nI have been thinking about our SOMU units, and I believe that we should focus on growing different species of kelp in each one. This will allow us to maximize the biodiversity of our kelp bed and provide a variety of habitats for marine life.\n\nI propose that we grow Giant Kelp in my SOMU, si

In [24]:
buoy_result = httpx.get("https://www.ndbc.noaa.gov/data/latest_obs/46092.rss")

In [25]:
print(buoy_result.text)

<?xml version="1.0"?>
<?xml-stylesheet type="text/xsl" href="/rss/ndbcrss.xsl"?>
<rss version="2.0" xmlns:georss="http://www.georss.org/georss" xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:atom="http://www.w3.org/2005/Atom">
  <channel>
    <title>NDBC - Station 46092 - MBM1 Observations</title>
    <description><![CDATA[This feed shows recent marine weather observations from Station 46092.]]></description>
    <link>https://www.ndbc.noaa.gov/</link>
    <pubDate>Tue, 30 Apr 2024 21:41:07 +0000</pubDate>
    <lastBuildDate>Tue, 30 Apr 2024 21:41:07 +0000</lastBuildDate>
    <ttl>30</ttl>
    <language>en-us</language>
    <managingEditor>webmaster.ndbc@noaa.gov (NDBC Webmaster)</managingEditor>
    <webMaster>webmaster.ndbc@noaa.gov (NDBC Webmaster)</webMaster>
    <image>
      <url>https://www.ndbc.noaa.gov/images/noaa_nws_xml_logo.gif</url>
      <title>NDBC - Station 46092 - MBM1 Observations</title>
      <link>https://www.ndbc.noaa.gov/</link>
    </image>
    <atom:link hre

In [27]:
for drone in [drone1, drone2, drone3]:
    time_stamp = datetime.datetime.now(datetime.UTC)
    drone.sensors["Current"] = Current(last_reading=time_stamp, value=21.4)
    drone.sensors["Depth"] = Depth(last_reading=time_stamp, value=2)
    for somu in drone.somus:
        o2 = chem_sensors.Oxygen(last_reading=time_stamp, value=-1)
        o2.value = random.randint(int(o2.suggested_range.minimum),
                                  int(o2.suggested_range.maximum))
        somu.sensors["O"] = o2
        light =  Light(last_reading=time_stamp, value=-1)
        light.value = random.randint(1,5)
        somu.sensors["Light"] = light
        somu.sensors["Temperature"] = Temperature(last_reading=time_stamp, value=10.8)

In [28]:
for drone in [drone1, drone2, drone3]:
    asyncio.ensure_future(drone.ooda_loop.run())

Aqua Observe: Sensor Data {'buoyancy': 0.0, 'current': 21.4, 'depth': 2, 'nutrient_levels': {'Carbon Dioxide': CarbonDioxide(last_reading=datetime.datetime(2024, 4, 30, 23, 9, 51, 615094, tzinfo=datetime.timezone.utc), value=-1, name='Carbon Dioxide', symbol='CO2', suggested_range=Range(minimum=0.1, maximum=2.0, scale=<Scale.mM: 'Millimolar'>)), 'Iron': Iron(last_reading=datetime.datetime(2024, 4, 30, 23, 9, 51, 615094, tzinfo=datetime.timezone.utc), value=-1, name='Iron', symbol='Fe', suggested_range=Range(minimum=0.1, maximum=2.0, scale=<Scale.ug_L: 'Micrograms per Liter'>)), 'Nitrogen': Nitrogen(last_reading=datetime.datetime(2024, 4, 30, 23, 9, 51, 615095, tzinfo=datetime.timezone.utc), value=-1, name='Nitrogen', symbol='N', suggested_range=Range(minimum=50, maximum=300, scale=<Scale.ug_L: 'Micrograms per Liter'>)), 'Oxygen': Oxygen(last_reading=datetime.datetime(2024, 4, 30, 23, 9, 51, 615096, tzinfo=datetime.timezone.utc), value=8, name='Oxygen', symbol='O2', suggested_range=Rang

In [30]:
drone1.somus[0].sensors["Temperature"]

Temperature(last_reading=datetime.datetime(2024, 4, 30, 23, 9, 51, 617605, tzinfo=datetime.timezone.utc), value=10.8, name='Temperature Sensor', scale=<Scale.celsius: 'Celsius'>)